In [1]:
import requests
import altair as alt
import pandas as pd
from pandas.io.json import json_normalize

from pyvis.network import Network
from ipywidgets import interact, fixed

import re

%load_ext autoreload
%autoreload 2
%cd ../intervals
from main import *
from main_objs import *
import visualizations as viz

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


## Relationships heatmaps

In [2]:
df_relationships = pd.read_csv('../CRIM_Intervals_Notebooks/relationships.csv')
files = df_relationships['model_observation.piece.piece_id'].unique()
df_relationships_0011 = df_relationships[df_relationships['model_observation.piece.piece_id'] == files[0]].copy()

In [3]:
df_relationships_0011.head()

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
0,0,http://crimproject.org/data/relationships/1/,1,Quotation,Fuga,True,True,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...",http://crimproject.org/data/observations/2/,2,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6/1,1,1+3,1+3,3,3/@1-4,@1-3,@1-3+@1-4,@1-3+@..."
1,1,http://crimproject.org/data/relationships/2/,2,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/4/,4,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-8/1,1+4,1+4,4/@3,@1-4+@4,@1+@1-3,@1"
2,2,http://crimproject.org/data/relationships/3/,3,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9,14-18/1,1,1-2,1-2,1-2,3,3,3-4,3-4,3-4/@3,@...",http://crimproject.org/data/observations/6/,6,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_5,Missa Tota pulchra es: Agnus Dei,"9-12/3-4,2-4,2,2/@4+@1-4,@4+@1-4+@1-3,@1-3,@1"
3,3,http://crimproject.org/data/relationships/4/,4,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/8/,8,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-10/1,1+4,1+4,1+4,1+4,4/@3,@1-4+@4,@1-4.5+@1-..."
183,183,http://crimproject.org/data/relationships/184/,184,Non-mechanical transformation,Imitative duo,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-7,10-16/1,1,1-2,1-2,2,2,2,3,3,3-4,3-4,4,4,4/...",http://crimproject.org/data/observations/368/,368,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6,10-13/1,1,1+3,1+3,3,3,2,2-3,2-3,3/@1-4,@1-..."


In [4]:
df_relationships_0011[df_relationships_0011['relationship_type']== "Quotation, Mechanical transformation"]

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
256,256,http://crimproject.org/data/relationships/258/,258,"Quotation, Mechanical transformation",Fuga,True,False,True,True,True,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"76-77/1+3,1+3/@3-4.5+@4,@1-2+@1-2.5",http://crimproject.org/data/observations/516/,516,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"85-87/1,1-2,2/@4,@1-3+@2-4.5,@1"


In [5]:
df_relationships_0011.iloc[0, :]

Unnamed: 0                                                                                 0
url                                             http://crimproject.org/data/relationships/1/
id                                                                                         1
relationship_type                                                                  Quotation
musical_type                                                                            Fuga
rt_q                                                                                    True
rt_q_x                                                                                  True
rt_q_monnayage                                                                         False
rt_tm                                                                                  False
rt_tm_snd                                                                              False
rt_tm_minv                                                            

In [6]:
viz.plot_comparison_heatmap(df_relationships_0011, 'model_observation.ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

['Fuga', 'Fuga', 'Fuga', 'Fuga', 'Fuga', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Fuga', 'Cadence', 'Fuga', 'Fuga', 'Fuga', 'Fuga', 'Homorhythm', 'Fuga', 'Homorhythm', 'Homorhythm', 'Homorhythm', 'Soggetto', 'Soggetto', 'Contrapuntal duo, Fuga', 'Fuga', 'Imitative duo', 'Fuga', 'Fuga', 'Fuga', 'Fuga', 'Form and process', 'Form and process', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Cantus firmus', 'Imitative duo', 'Imitative duo', 'Fuga', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Fuga', 'Fuga', 'Fuga', 'Fuga', 'Fuga', 'Imitative duo', 'Imitative duo', 'Fuga', 'Fuga', 'Fuga', 'Imitative duo', 'Imitative duo', 'Fuga', 'Fuga', 'Fuga', 'Fuga', 'Soggetto', 'Soggetto', 'Homorhythm', 'Homorhythm', 'Fuga', 'Soggetto', 'Soggetto', 'Soggetto', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Imitative duo', 'Fuga', 'Fuga', 'Imitative duo', 'Imitative duo', 'Fuga', 'Fuga', 'Sogge

alt.VConcatChart(...)

In [7]:
#retrieve_notes_and_durations('https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/CRIM_Model_0011.mei', '1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,@1-3,@1')

## Observations heatmap

In [8]:
df_observations = pd.read_csv('../CRIM_Intervals_Notebooks/observations.csv')

In [9]:
df_observations_pen = df_observations[df_observations['musical_type']=='Periodic entry'].copy()
df_observations_pen.iloc[0, :]

Unnamed: 0                                                          5
url                       http://crimproject.org/data/observations/6/
id                                                                  6
ema                     9-12/3-4,2-4,2,2/@4+@1-4,@4+@1-4+@1-3,@1-3,@1
musical_type                                           Periodic entry
                                          ...                        
observer.name                                            Drew Ivarson
piece.url           http://crimproject.org/data/pieces/CRIM_Mass_0...
piece.piece_id                                       CRIM_Mass_0009_5
piece.full_title                     Missa Tota pulchra es: Agnus Dei
piece.mass                                             CRIM_Mass_0009
Name: 5, Length: 92, dtype: object

In [10]:
networks, interactive_df = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

interactive(children=(Text(value='', description='search_pattern_starts_with'), Output()), _dom_classes=('widg…

In [11]:
#networks['5-'].show('../CRIM_Intervals_Notebooks/test.html')

## Ngrams heatmaps

In [12]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams

Successfully imported.


,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,"1, 1, 1, 2, 1","1, 3, -3, 8, -2",NaN,NaN,NaN,NaN
8.0,"1, 1, 2, 1, 1","3, -3, 8, -2, -3",NaN,NaN,NaN,NaN
12.0,"1, 2, 1, 1, 1","-3, 8, -2, -3, 3",NaN,NaN,NaN,NaN
16.0,"2, 1, 1, 1, 2","8, -2, -3, 3, 2",NaN,NaN,NaN,NaN
20.0,"1, 1, 1, 2, 1","-2, -3, 3, 2, 1",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1508.0,"1, 1, 1, 1, 1","1, 1, 1, 1, 1","5, 1, 1, 1, 1","-3, 8, 1, 1, 1","1, 1, 1, 1, 1","-3, -3, 1, 1, 1"
1512.0,"1, 1, 1, 1, 2","1, 1, 1, 1, 2","1, 1, 1, 1, -2","8, 1, 1, 1, -3","1, 1, 1, 1, -5","-3, 1, 1, 1, 4"
1516.0,"1, 1, 1, 2, 1","1, 1, 1, 2, 1","1, 1, 1, -2, 1","1, 1, 1, -3, 1","1, 1, 1, -5, 1","1, 1, 1, 4, 1"
1520.0,"1, 1, 2, 1, 1","1, 1, 2, 1, 1","1, 1, -2, 1, 1","1, 1, -3, 1, 1","1, 1, -5, 1, 1","1, 1, 4, 1, 1"


In [13]:
mel_ngram_list = mel_ngrams.stack().dropna().value_counts().head(10).index.to_list()

In [14]:
# viz.plot_close_match_heatmap(mel_ngrams, "-P1, -P1, -P1, M2", selected_patterns=mel_ngram_list, voices=[])

In [15]:
viz.plot_close_match_heatmap(mel_ngrams, "1, 2, 1, -3", selected_patterns=mel_ngram_list, voices=[])

alt.Chart(...)

## Heatmaps with for ngrams!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

In [16]:
nr = model.getNoteRest()
dur = model.getDuration(nr.applymap(str, na_action='ignore'))

## Weighted Levenshtein

In [17]:
from strsimpy.weighted_levenshtein import WeightedLevenshtein

def insertion_cost(char):
    return 1.0


def deletion_cost(char):
    return 1.0


def substitution_cost(char_a, char_b):
    return abs(char_a - char_b)

weighted_levenshtein = WeightedLevenshtein(
    substitution_cost_fn=substitution_cost,
    insertion_cost_fn=insertion_cost,
    deletion_cost_fn=deletion_cost)

In [18]:
model.getMelodic(kind='q', directed=True, compound=True, unit=0)

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,NaN,NaN,Rest,Rest,Rest,Rest
4.0,NaN,P1,NaN,NaN,NaN,NaN
7.0,NaN,M2,NaN,NaN,NaN,NaN
8.0,P1,M2,Rest,Rest,Rest,Rest
10.0,NaN,m2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1508.0,NaN,NaN,M2,-P5,NaN,m3
1510.0,NaN,NaN,-P5,NaN,NaN,NaN
1511.0,NaN,NaN,NaN,NaN,NaN,-m3
1512.0,NaN,NaN,P5,P8,NaN,P1


In [19]:
df_relationships.iloc[26, :]

Unnamed: 0                                                                                26
url                                            http://crimproject.org/data/relationships/27/
id                                                                                        27
relationship_type                                   Quotation, Non-mechanical transformation
musical_type                                                                Counter-soggetto
rt_q                                                                                    True
rt_q_x                                                                                  True
rt_q_monnayage                                                                         False
rt_tm                                                                                  False
rt_tm_snd                                                                              False
rt_tm_minv                                                            

In [20]:
nt = viz.plot_relationship_network(df_relationships, selected_model_ids=['CRIM_Model_0008', 'CRIM_Model_0011'], color='derivative')
# %cd ../CRIM_Intervals_Notebooks
nt.show('0011.html')

In [28]:
df_relationships['model_observation.piece.piece_id'].unique()

array(['CRIM_Model_0011', 'CRIM_Model_0018', 'CRIM_Model_0017',
       'CRIM_Mass_0015_2', 'CRIM_Model_0001', 'CRIM_Model_0020',
       'CRIM_Model_0009', 'CRIM_Model_0019', 'CRIM_Model_0010',
       'CRIM_Model_0022', 'CRIM_Model_0002', 'CRIM_Model_0016',
       'CRIM_Model_0024', 'CRIM_Model_0021', 'CRIM_Mass_0018_1',
       'CRIM_Model_0008', 'CRIM_Mass_0018_3', 'CRIM_Model_0012',
       'CRIM_Model_0023', 'CRIM_Model_0015', 'CRIM_Model_0013',
       'CRIM_Mass_0005_1', 'CRIM_Mass_0016_2', 'CRIM_Mass_0020_3',
       'CRIM_Model_0006', 'CRIM_Model_0025'], dtype=object)

In [22]:
dir(viz)

['Fraction',
 'Network',
 'RELATIONSHIP_WEIGHTS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_close_match',
 '_close_match_helper',
 '_from_ema_to_offsets',
 '_manipulate_processed_network_df',
 '_plot_ngrams_df_heatmap',
 '_process_crim_json_url',
 '_process_ngrams_df_helper',
 '_recognize_integers',
 '_trim_and_combine_piece_ids_with_measures',
 'alt',
 'create_bar_chart',
 'create_comparisons_networks_and_interactive_df',
 'create_heatmap',
 'create_interactive_compare_df',
 'create_interval_networks',
 'fixed',
 'generate_ngrams_and_duration',
 'group_observations',
 'interact',
 'pd',
 'plot_close_match_heatmap',
 'plot_comparison_heatmap',
 'plot_ngrams_heatmap',
 'plot_relationship_network',
 'process_network_df',
 'process_ngrams_df',
 're',
 'textdistance']

In [34]:
nt2 = viz.plot_relationship_network(df_relationships, selected_derivative_ids=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt2.show('../CRIM_Intervals_Notebooks/selected_family.html')

In [41]:
nt3 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002', 'CRIM_Model_0016'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('../CRIM_Intervals_Notebooks/selected_model_family.html')

5 CRIM_Mass_0017_3, CRIM_Mass_0008_4, CRIM_Mass_0018_1, CRIM_Mass_0018_2, CRIM_Mass_0005_5 no longer exist in df because of other filtering options


In [36]:
nt4 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002', 'CRIM_Model_0016'], color='derivative')
nt4.show('../CRIM_Intervals_Notebooks/selected_model.html')

In [40]:
nt3 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002', 'CRIM_Model_0016'], selected_derivative_ids=['CRIM_Mass_0003_5', 'CRIM_Mass_0014'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('../CRIM_Intervals_Notebooks/selected_model_deriv_family.html')

6 CRIM_Mass_0017_3, CRIM_Mass_0008_4, CRIM_Mass_0014_4, CRIM_Mass_0018_1, CRIM_Mass_0018_2, CRIM_Mass_0005_5 no longer exist in df because of other filtering options
